# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow

tensorflow==2.10.0
tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2024-02-10 16:59:29.161910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 16:59:29.517179: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-10 16:59:29.517200: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-02-10 16:59:29.566924: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 16:59:30.810045: W tensorflow/stream_executor/platform/de

<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [5]:
# YOUR CODE HERE
# wv['crocodile']

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [6]:
# YOUR CODE HERE
wv['blue']

array([-9.37514305e-02,  1.48005098e-01, -8.62244368e-02,  1.55332983e-01,
        1.37290871e-02, -2.78825253e-01,  7.56460577e-02,  4.49860722e-01,
       -1.75621867e-01, -1.45509794e-01,  9.09184106e-03, -2.81677723e-01,
        1.00247867e-01,  1.58443555e-01, -2.64830119e-03, -1.84567064e-01,
        2.19598472e-01, -1.88038409e-01, -2.50500645e-02, -3.31595451e-01,
        1.50085002e-01,  3.34893279e-02,  2.69007832e-01, -1.09927274e-01,
        3.06198169e-02,  1.69417337e-02, -2.12716684e-01, -6.99582025e-02,
       -1.15444832e-01, -4.36544828e-02,  2.52055377e-01,  8.00242182e-03,
        1.18556388e-01, -2.62557060e-01, -1.65684998e-01,  2.35164464e-01,
        6.46145269e-02, -9.25241634e-02, -1.96354613e-01, -3.40491086e-01,
       -1.10602938e-01, -2.28268370e-01, -1.18279010e-01,  1.28636837e-01,
        1.77981466e-01,  8.37802887e-03, -1.80532038e-01, -8.24238211e-02,
        1.41395390e-01,  1.39883682e-01,  3.98077555e-02, -1.31458566e-01,
       -1.96525604e-01, -

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [7]:
# YOUR CODE HERE#

wv.most_similar('dog', topn=10) # topn shows just 10 closest synonyms

[('using', 0.990145206451416),
 ('naked', 0.9883840680122375),
 ('water', 0.9880982637405396),
 ('ugly', 0.9880604147911072),
 ('vice', 0.9871379733085632),
 ('leaving', 0.9868494272232056),
 ('conan', 0.9857728481292725),
 ('mayhem', 0.9857625961303711),
 ('rising', 0.9856146574020386),
 ('news', 0.9852110147476196)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [8]:
# YOUR CODE HERE

wv.similar_by_vector('dog')

[('using', 0.990145206451416),
 ('naked', 0.9883840680122375),
 ('water', 0.9880982637405396),
 ('ugly', 0.9880604147911072),
 ('vice', 0.9871379733085632),
 ('leaving', 0.9868494272232056),
 ('conan', 0.9857728481292725),
 ('mayhem', 0.9857625961303711),
 ('rising', 0.9856146574020386),
 ('news', 0.9852110147476196)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [9]:
# YOUR CODE HERE
wv['good'] - wv['bad']

array([-0.14145994, -0.28932676,  0.34609097,  0.1335833 , -0.21614954,
       -0.41691744, -0.2671221 ,  0.09561204, -0.26712072, -0.5152265 ,
        0.01156279,  0.30807576,  0.33220604,  0.06971025,  0.169848  ,
        0.50596035, -0.4209203 ,  0.286481  , -0.20638868,  0.05941832,
       -0.04627645,  0.18584672,  0.11806226, -0.09609832,  0.12868915,
       -0.24318764, -0.20965423,  0.47034943, -0.06634478, -0.5303607 ,
       -0.08871937,  0.3073794 ,  0.13008338,  0.18656644, -0.25239375,
       -0.4051025 ,  0.49954098, -0.10180569, -0.37610427,  0.9849169 ,
       -0.14399278,  0.256665  , -0.42495316, -0.0387001 ,  0.39034897,
       -0.06654701,  0.4929223 , -0.18479307, -0.1454556 ,  0.43180877,
        0.04012161, -0.33294895, -0.68586737,  0.10825595,  0.25401354,
        0.26734895, -0.01509118,  0.44495374,  0.2614985 ,  0.40828907,
       -0.07695106,  0.48025292, -0.30655444, -0.17206007,  0.11294055,
        0.20992053, -0.41988227, -0.41158321,  0.41073775,  0.40

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [10]:
# YOUR CODE HERE
res = wv['good'] - wv['bad'] + wv['stupid']

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [11]:
# YOUR CODE HERE

wv.similar_by_vector(res)

[('good', 0.7953722476959229),
 ('great', 0.7745206952095032),
 ('nice', 0.7685246467590332),
 ('strongest', 0.7496516704559326),
 ('also', 0.7438157796859741),
 ('done', 0.7357857823371887),
 ('decent', 0.73538738489151),
 ('gory', 0.719541072845459),
 ('criticized', 0.717643141746521),
 ('used', 0.717042088508606)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [12]:
# YOUR CODE HERE

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [13]:
# YOUR CODE HERE
word2vec_2 = Word2Vec(sentences=X_train, vector_size = 50)
wv2 = word2vec_2.wv
len(wv2['dog'])

50

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [14]:
# YOUR CODE HERE

vocab_size = len(wv.key_to_index)
vocab_size

8006

In [15]:
num_unique_words_in_X_train = len(set([word for sentence in X_train for word in sentence]))
num_unique_words_in_X_train

30419

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [16]:
# YOUR CODE HERE

word2vec_3 = Word2Vec(sentences=X_train, min_count=10) # bigger min_count, less vocab analysed.
wv3 = word2vec_3.wv
vocab_size = len(wv3.key_to_index)
vocab_size

4503

In [17]:
word2vec_4 = Word2Vec(sentences=X_train, min_count=2)  # smaller min_count, more vocab analysed.
wv4 = word2vec_4.wv
vocab_size = len(wv4.key_to_index)
vocab_size

16729

Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [18]:
# YOUR CODE HERE
word2vec_5 = Word2Vec(sentences=X_train, window = 20)# Maximum distance between the current and predicted word within a sentence.
wv5 = word2vec_5.wv
vocab_size = len(wv5.key_to_index)
vocab_size

8006

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [19]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
    return np.array(embedded_sentence)
    
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

In [20]:
embedded_sentence = embed_sentence(word2vec, example)
print(embedded_sentence.shape)

(8, 100)


❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [22]:
def embedding(word2vec, sentences):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embedded_sentences.append(embedded_sentence)
    return embedded_sentences
    
X_train_embedded = embedding(word2vec, X_train)
X_test_embedded = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [ ]:
### YOUR CODE HERE

from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the training and test sequences

max_length = max(len(sentence) for sentence in X_train)
max_length
X_train_pad = pad_sequences(X_train_embedded, maxlen=max_length, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_embedded, maxlen=max_length, dtype='float32', padding='post')


# assert(len(X_train_pad.shape) == 3)
# assert(len(X_test_pad.shape) == 3)
# assert(X_train_pad.shape[2] == 100)
# assert(X_test_pad.shape[2] == 100)

NameError: name 'X_train_pad' is not defined



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
